In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import json
from pandas import json_normalize
# pd.__version__

In [2]:
topic_file = '/Users/bew/Downloads/DataForThai_jupyter/DataForThai_Pandas&SubTopic/SubTopic_dataforthai.csv'
topic_table = pd.read_csv(topic_file)


topic_table['รหัส'] = topic_table['รหัส'].astype(str)
# Function to add '0' to strings of length 4
def add_zero(s):
    if len(s) == 4:
        return '0' + s
    else:
        return s
# Applying the function to the DataFrame
topic_table['รหัส'] = topic_table['รหัส'].apply(lambda x: add_zero(x))


# topic_table.dtypes
topic_table = topic_table[:132] # 10211 # update index (ไม่จำเป็นต้อง Up แค่ไม่ให้ Run เกินต้องการ)
# topic_table = topic_table[topic_table['รหัส']=='01450']
# print(topic_table)
# max(topic_table['จำนวนหน้า'])
topic_table

,รหัส,ประเภทธุรกิจ,จำนวน,จำนวนหน้า
0,01111,การปลูกข้าวโพดที่ใช้เมล็ดแก่,24,1
1,01112,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),34,1
2,01113,การปลูกพืชตระกูลถั่ว,15,1
3,01114,การปลูกถั่วเหลือง,3,1
4,01115,การปลูกพืชน้ำมัน (ยกเว้นถั่วเหลือง),10,1
...,...,...,...,...
127,10133,การผลิตเนื้อสัตว์และเนื้อสัตว์ปีกบรรจุกระป๋อง,6,1
128,10134,การผลิตผลิตภัณฑ์พลอยได้จากสัตว์และสัตว์ปีก,91,1
129,10139,การผลิตผลิตภัณฑ์อื่น ๆ จากเนื้อสัตว์และเนื้อสั...,184,1
130,10210,การผลิตสัตว์น้ำและผลิตภัณฑ์สัตว์น้ำสดแช่เย็นหร...,1,1


In [3]:
# topic_for_file = topic_table[topic_table['รหัส'] == no_topic]
# topic_for_file

In [4]:
# # Read JSON file
# file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/01111.json'  # Replace with your file path

# df = pd.read_json(file_path)

# # # Load JSON data into DataFrame
# # df = json_normalize(json_data)  # Use pd.DataFrame(json_data) if the JSON is already flat

# # df['สถานะ'] = df['สถานะ'].astype("string")
# df.dtypes
# # df

In [5]:
import pandas as pd
import os

# Initialize an empty DataFrame for the result
result = pd.DataFrame()

lastId = "10211" # update 'lastID' เพื่อไม่อ่านข้อมูลเดิม เพราะค้นหาข้อมูลมาทับ
value = ""

for index, row in topic_table.iterrows():
    
    value1 = row['รหัส']
    value2 = row['ประเภทธุรกิจ']
    
    if (int(value1) <= int(lastId)): continue
    
    # Construct the file path
    file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/{value1}.json'
    
    # Check if the file exists
    if os.path.exists(file_path):
        try:
            # Read JSON file
            df = pd.read_json(file_path)
            
            # Add a new column with the business type
#             df['ประกอบธุรกิจ'] = value2
            
            # Concatenate the current DataFrame to the result DataFrame
            result = pd.concat([result, df], axis=0)
        
        except ValueError as e:
            print(f"Error reading JSON file at {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")
    value = value1

print(f"lastId = {value1}")

lastId = 10211


In [6]:
# Reset the index of the final result DataFrame
result = result.reset_index(drop=True)

# result

In [7]:
result.columns.tolist()

result['เบอร์โทร'] = np.nan
# new_order = ['เลขทะเบียน', 'ชื่อบริษัทภาษาไทย', 'ชื่อบริษัทภาษาอังกฤษ', 'ประกอบธุรกิจ', 'หมวดธุรกิจ', 'สถานะ',
#              'วันที่จดทะเบียน', 'ทุนจดทะเบียน', 'ที่ตั้ง', 'ข้อมูลสำหรับการติดต่อ', 'เบอร์โทร', 'Website', 'ก่อตั้งโดย',
#              'หลักทรัพย์', 'ที่มา', 'สมัครงาน / ดูตำแหน่งงานว่าง', 'วันที่ร้าง', 'วันที่เลิก']
# result = result[new_order]
# result

In [8]:
# for index, row in topic_table.iterrows():
    
#     value1 = row['รหัส']
#     value2 = row['ประเภทธุรกิจ']
    
#     # Read JSON file
#     file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/{value1}.json'  # Replace with your file path

#     df = pd.read_json(file_path)
    
#     df['ประกอบธุรกิจ'] = value2
    
#     result = pd.concat([result, df], axis=0)
    
    
# result


In [9]:
# result = result.drop(columns=['สมัครงาน / ดูตำแหน่งงานว่าง', 'วันที่ร้าง', 'หลักทรัพย์'])

result = result.applymap(str)
# result['เลขทะเบียน'] = result['เลขทะเบียน'].map(str)
# result['สถานะ'] = result['สถานะ'].astype("string")


# Step 1: Remove commas from the strings
result['ทุนจดทะเบียน'] = result['ทุนจดทะเบียน'].str.replace(',', '')
result['ทุนจดทะเบียน'] = result['ทุนจดทะเบียน'].str.replace(' บาท', '')
# Step 2: Convert the strings to numeric type (int or float)
result['ทุนจดทะเบียน'] = pd.to_numeric(result['ทุนจดทะเบียน'])

result.dtypes

KeyError: 'ทุนจดทะเบียน'

In [ ]:
result = result[result['สถานะ']=='ยังดำเนินกิจการอยู่']
result = result[(result['ทุนจดทะเบียน'] >= 30000000)]

# def get_length(obj):
#     try:
#         return len(obj)
#     except TypeError:
#         return None

# for text in result['ทุนจดทะเบียน']:
#     print(text.apply(get_length))

# for index, row in result.iterrows():
    
#     value1 = row['เลขทะเบียน']
#     if value1.apply(len) != 13:
#         value1 = "0" + value1.astype(str)

result['เลขทะเบียน'] = "'0" + result['เลขทะเบียน'].astype(str)
# # Function to add '0' to strings of length 13
# def add_zero(s):
# #     s = str(s)  # Ensure the input is a string
#     if len(s) == 14:
#         return '0' + s
#     else:
#         return s

# # Applying the function to the DataFrame
# result['เลขทะเบียน'] = result['เลขทะเบียน'].apply(lambda x: add_zero(x))

# # Reset the index of the final result DataFrame
# result = result.reset_index(drop=True)

# result.dtypes

In [ ]:
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มกราคม ','/1/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กุมภาพันธ์ ','/2/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มีนาคม ','/3/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' เมษายน ','/4/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' พฤษภาคม ','/5/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มิถุนายน ','/6/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กรกฎาคม ','/7/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' สิงหาคม ','/8/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กันยายน ','/9/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' ตุลาคม ','/10/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' พฤศจิกายน ','/11/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' ธันวาคม ','/12/')

In [ ]:
result = result.reset_index(drop=True)
# result

In [ ]:
# result.dtypes

In [ ]:
file = '/Users/bew/Downloads/DataForThai_jupyter/FilterAndSearchData.csv'
df = pd.read_csv(file)

df_updated = pd.concat([df, result], ignore_index=True)
df_updated = df_updated.drop_duplicates(subset='เลขทะเบียน')
df_updated = df_updated.reset_index(drop=True)

df_updated.dtypes
df_updated

In [ ]:
# result[result['เลขทะเบียน'].str.len() != 13]

In [ ]:
# result

In [ ]:
file_name = 'FilterAndSearchData.csv'
file_path = '/Users/bew/Downloads/DataForThai_jupyter/'

# Combine the file path and file name
full_path = os.path.join(file_path, file_name)

# result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8') # รอบ 1
df_updated.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8') # รอบ 2+

print('finish')

In [ ]:
# # Step 1: Remove commas from the strings
# df['Capital'] = df['Capital'].str.replace(',', '')

# # Step 2: Convert the strings to numeric type (int or float)
# df['Capital'] = pd.to_numeric(df['Capital'])

# # Display the DataFrame to verify the changes
# df